In [10]:
import tensorflow as tf
from tensorflow.python import keras
import numpy as np
import process
from file_handler import File_Handler
from sklearn.model_selection import train_test_split

## Verify version

In [11]:
print(keras.__version__)
print(tf.__version__)

2.2.4-tf
2.0.0


## Global Variables

In [12]:
malware_path = "../data/malware/"
benign_path = "../data/benign/"
keep_amt = 30
last_key = "other"

max_opcode_len = 10000
embedding_vector_len = 64
lstm_num_units = 150
batch_size = 64
num_epochs = 20
test_size= 0.2
checkpoint_path = 'training_checkpoint.keras'
log_dir = 'logs/'

In [13]:
mal_handler, ben_handler, most_common_opcodes = process.get_most_common_opcodes(
                                                                    keep_amt = keep_amt, 
                                                                    last_key = last_key, 
                                                                    malware_path = malware_path,
                                                                    benign_path = benign_path)

longest_len = max(mal_handler.get_longest_opcode_seq(), ben_handler.get_longest_opcode_seq())

print(most_common_opcodes)
print(longest_len)

{'mov': 0, 'push': 1, 'add': 2, 'call': 3, 'cmp': 4, 'jmp': 5, 'xor': 6, 'pop': 7, 'jz': 8, 'jnz': 9, 'lea': 10, 'sub': 11, 'test': 12, 'retn': 13, 'or': 14, 'and': 15, 'inc': 16, 'nop': 17, 'dec': 18, 'shr': 19, 'movzx': 20, 'jb': 21, 'sbb': 22, 'adc': 23, 'shl': 24, 'leave': 25, 'imul': 26, 'jnb': 27, 'jbe': 28, 'xchg': 29, 'other': 30}
360169


In [14]:
def populate_train_set(file_handler):
    num_files = file_handler.get_num_files()
    train_set = list()

    # populate train_set
    for file_counter,file in enumerate(file_handler.get_files()):
        opcode_counter = 0
        with open(file, 'r') as f:
            file_opcodes = list()
            opcode = f.readline()
            while opcode and opcode_counter < max_opcode_len:
                opcode = opcode.strip()
                if opcode in most_common_opcodes:
                    opcode_key = most_common_opcodes[opcode]
                else:
                    opcode_key = most_common_opcodes[last_key]
                
                file_opcodes.append(opcode_key)
                opcode_counter += 1
                opcode = f.readline()
        train_set.append(file_opcodes)

    return train_set

In [15]:
# labels: 1 for malware, 0 for benign

mal_train_set = populate_train_set(mal_handler)
mal_labels = np.ones( shape=(len(mal_train_set),1) )

ben_train_set = populate_train_set(ben_handler)
ben_labels = np.zeros( shape=(len(ben_train_set),1) )

mal_train_set = mal_train_set[:200]
mal_labels = mal_labels[:200]

train_set = np.concatenate((mal_train_set, ben_train_set), axis=0)
train_set = tf.keras.preprocessing.sequence.pad_sequences(train_set, maxlen=max_opcode_len)
train_labels = np.concatenate((mal_labels, ben_labels), axis=0)

train_set, test_set, train_labels, test_labels = train_test_split(train_set, train_labels, test_size=test_size)

print(len(mal_train_set))
print(len(ben_train_set))
print("train_set shape: {}".format(train_set.shape))
print("train_labels shape: {}".format(train_labels.shape))
print("test_set shape: {}".format(test_set.shape))
print("test_labels shape: {}".format(test_labels.shape))

200
130
train_set shape: (264, 10000)
train_labels shape: (264, 1)
test_set shape: (66, 10000)
test_labels shape: (66, 1)


## Make the model

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=keep_amt+1,
                                    output_dim=embedding_vector_len,
                                    input_length=max_opcode_len))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(lstm_num_units))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


optimizer = tf.keras.optimizers.Adam()

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [17]:
model.fit(x=train_set,
          y=train_labels,
          batch_size=batch_size,
          epochs=num_epochs,)

Train on 264 samples
Epoch 1/20
264/264 [==============================] - 4s 16ms/sample - loss: 0.6840 - accuracy: 0.6136
Epoch 2/20
264/264 [==============================] - 2s 9ms/sample - loss: 0.6476 - accuracy: 0.7879
Epoch 3/20
264/264 [==============================] - 2s 8ms/sample - loss: 0.7530 - accuracy: 0.7273
Epoch 4/20
264/264 [==============================] - 2s 8ms/sample - loss: 0.4278 - accuracy: 0.8750
Epoch 5/20
264/264 [==============================] - 2s 8ms/sample - loss: 0.3463 - accuracy: 0.8902
Epoch 6/20
264/264 [==============================] - 2s 9ms/sample - loss: 0.2404 - accuracy: 0.9508
Epoch 7/20
264/264 [==============================] - 2s 8ms/sample - loss: 0.2885 - accuracy: 0.8977
Epoch 8/20
264/264 [==============================] - 2s 8ms/sample - loss: 0.2321 - accuracy: 0.9659
Epoch 9/20
264/264 [==============================] - 2s 8ms/sample - loss: 0.4797 - accuracy: 0.8295
Epoch 10/20
264/264 [==============================] - 2s 8m

## Test the model

In [18]:
scores = model.evaluate(test_set, test_labels, verbose=0)
print("Accuracy: %0.2f%%" % (scores[1]*100))

Accuracy: 96.97%
